<center><img src="images/DLI Header.png" alt="Header" width="400"></center>

# 3.0 Multiple Networks Application
Once an object is detected and located in a DeepStream app, it can be further classified by passing the cropped object image through additional network(s) in the pipeline.  Those additional networks can run image classification inference to provide _more information_ about the objects. In this notebook, you'll work with the `deepstream-test2` reference application to find objects in a video stream, pass those images through a series of classification networks, and finally display detailed information about the objects in the output stream.

<img src="images/02_test2_example.png" alt="beginning and end image with pipe">

**[3.1 Build a Pipeline with Multiple Networks in Series](#3.1-Build-a-Pipeline-with-Multiple-Networks-in-Series)**<br>
&nbsp; &nbsp; &nbsp;[3.1.1 Practice Application `deepstream-test2-rtsp-out-1SGIE`](#3.1.1-Practice-Application-deepstream-test2-rtsp-out-1SGIE)<br>
&nbsp; &nbsp; &nbsp;[3.1.2 Secondary Networks](#3.1.2-Secondary-Networks)<br>
&nbsp; &nbsp; &nbsp;[3.1.3 Exercise: Run the Base Application](#3.1.3-Exercise:-Run-the-Base-Application)<br>
**[3.2 Add a Plugin to a Pipeline](#3.2-Add-a-Plugin-to-a-Pipeline)**<br>
&nbsp; &nbsp; &nbsp;[3.2.1 Instantiate and Set Properties](#3.2.1-Instantiate-and-Set-Properties)<br>
&nbsp; &nbsp; &nbsp;[3.2.2 Add to Pipeline and Link](#3.2.2-Add-to-Pipeline-and-Link)<br>
&nbsp; &nbsp; &nbsp;[3.2.3 Configure](#3.2.3-Configure)<br>
&nbsp; &nbsp; &nbsp;[3.2.4 Exercise: Add SGIE2 Plugin to the Pipeline](#3.2.4-Exercise:-Add-SGIE2-Plugin-to-the-Pipeline)<br>
**[3.3 Put It All Together](#3.3-Put-It-All-Together)**<br>
&nbsp; &nbsp; &nbsp;[3.3.1 Exercise: Three Secondary Networks](#3.3.1-Exercise:-Three-Secondary-Networks)<br>

# 3.1 Build a Pipeline with Multiple Networks in Series
To enable multiple neural networks to reside within a single pipeline, we add one `Gst-nvtracker` plugin. Then we add multiple instances of the `Gst-nvinfer` plugin (in other words, multiple inference engines). Each `Gst-nvinfer` will build upon the results of the previous one. This results in the ability to identify sub-classifications of detected objects. 

The first instance of the `Gst-nvinfer` plugin serves as the **Primary GPU Inference Engine (PGIE)**. For example, we can start with a PGIE 4-class object detector that detects vehicles, bicycles, persons, and road signs. 

Subsequent instances of the `Gst-nvinfer` plugin are **Secondary GPU Inference Engines (SGIE)**. During object detection inference, a **region of interest (ROI)** is determined and vertices placed into the metadata. When an object ROI is detected by the PGIE, a link is generated by `Gst-nvtracker` to track the object between frames. The SGIE takes the ROI (in other words, the cropped image) as input, and provides an output with secondary identifying information.  

For example, the primary inference detector (PGIE) can locate "car" objects, then feed the ROI into a series of three secondary inference classifiers (SGIE1, SGIE2, SGIE3).  If these three classifiers infer color, make, and type, the annotation for a detected vehicle would be something like:

<img src="images/02_3sgie_label.png">

in accordance with the labels provided as part of the inference models.



## 3.1.1 Practice Application `deepstream-test2-rtsp-out-1SGIE`
The  DeepStream SDK `deepstream_test2` sample application pipeline includes a primary detector and three secondary detectors. For this notebook, you will start with a modified version in the `dli_python_apps` directory that includes the RTSP output and *only one* secondary network.  Execute the next cell to take a look at the application directory. Note that there are now three configuration files: two for the two `Gst-nvinfer` plugins, and one for the `Gst-nvtracker` plugin.

In [ ]:
# Set some path locations for readability
PYTHON_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/apps'
STREAMS = '/opt/nvidia/deepstream/deepstream/samples/streams'
DLI_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/dli_apps'
MY_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/my_apps'
# Make sure common utilities are in my_apps
!cp -r $PYTHON_APPS/common $MY_APPS/

# List the contents of the sample app
!ls $DLI_APPS/deepstream-test2-rtsp-out-1SGIE

You can view the Python code in [\$DLI_APPS/deepstream-test1-rtsp-out-1SGIE/deepstream_test2_rtsp_out.py](deepstream/sources/deepstream_python_apps/dli_apps/deepstream-test2-rtsp-out-1SGIE/deepstream_test2_rtsp_out.py).  Here's a look at a snippet showing the plugins added (shown in order) to the pipeline:

```Python
    print("Adding elements to Pipeline \n")
    pipeline.add(source)
    pipeline.add(h264parser)
    pipeline.add(decoder)
    pipeline.add(streammux)
    pipeline.add(pgie)
    pipeline.add(tracker)
    pipeline.add(sgie1)
    pipeline.add(nvvidconv)
    pipeline.add(nvosd)
# for RTSP ###################
    pipeline.add(nvvidconv_postosd)
    pipeline.add(caps)
    pipeline.add(encoder)
    pipeline.add(rtppay) 
######  
    pipeline.add(sink)
```

This DeepStream application includes the following plugins in its pipeline:

- `GstFileSrc` - reads the video data from file
- `GstH264Parse` - parses the incoming H264 stream
- `Gst-nvv4l2decoder` - hardware accelerated decoder; decodes video streams using NVDEC
- `Gst-nvstreammux` - batch video streams before sending for AI inference
- `Gst-nvinfer` - (PGIE) runs inference using TensorRT
- `Gst-nvtracker` - tracks object between frames
- `Gst-nvinfer` (SGIE1) - runs inference using TensorRT
- `Gst-nvvideoconvert` - performs video color format conversion (I420 to RGBA)
- `Gst-nvdsosd` - draw bounding boxes, text and region of interest (ROI) polygons
- `Gst-nvvideoconvert` - performs video color format conversion (RGBA to I420)
- `GstCapsFilter` - enforces limitations on data (no data modification)
- `Gst-nvv4l2h264enc` - encodes RAW data in I420 format to H264
- `GstRtpH264Pay` - converts H264 encoded Payload to RTP packets (RFC 3984)
- `GstUDPSink` - sends UDP packets to the network. When paired with RTP payloader (`Gst-rtph264pay`) it can implement RTP streaming

## 3.1.2 Secondary Networks
The pipeline for `deepstream-test2-rtsp-out-1SGIE` is very similar to the `deepstream_test1` object detection sample application you worked with previously. The only real difference is the addition of a tracker and secondary networks (only one secondary network to start with).  The diagram below gives an idea of how secondary networks fit into the pipeline after the primary detector.

<img src="images/02_secondary_networks.png" alt="secondary networks">

In `deepstream-test2-rtsp-out-1SGIE`, two plugins are inserted after the `Gst-nvinfer` object detector (PGIE): `Gst-nvtracker`, and one classification network using an additional `Gst-nvinfer` plugin (SGIE1). In a similar way, additional networks can be added to the pipeline by adding more `Gst-nvinfer` plugins to the pipeline (e.g. SGIE2, SGIE3).  You'll try this in a later exercise.  For now, try the example with a single secondary network that determines the color of vehicles found by the object detector.

## 3.1.3 Exercise: Run the Base Application

In [ ]:
# Check usage of the test2 app with the help option
!cd $DLI_APPS/deepstream-test2-rtsp-out-1SGIE \
    && python3 deepstream_test2_rtsp_out.py --help

#### Run the DeepStream app
If using VLC media player, open the app on your computer:
- Pull down the "Media" menu and select the "Open Network Stream" dialog.
- Set the URL to `rtsp://192.168.55.1:8554/ds-test`.
- Optionally, add a wait delay to VLC:
   - Click "Show more options" in the dialog.
   - Add ` :ipv4=120000` to the "Edit Options" line to add a 120 second delay.
- Start execution of the cell below.
- Click "Play" on your VLC media player *after* you start the cell execution.  

The stream will start from the Jetson Nano and display in the media player.  There is a delay while the model `.engine` file is built.  

If VLC fails, start it again. Close the VLC fail notice and press the "play" triangle.

In [ ]:
# Run the app
!cd $DLI_APPS/deepstream-test2-rtsp-out-1SGIE \
    && python3 deepstream_test2_rtsp_out.py -i $STREAMS/sample_720p.h264

# 3.2 Add a Plugin to a Pipeline

In the following snippets from `deepstream-test2-rtsp-out-1SGIE`, the code required for the SGIE1 plugin addition are highlighted.  Adding additional SGIE-n networks require the same steps: <br>
1. Instantiate Plugin and Set Properties
1. Add Plugin to Pipeline and Link
1. Configure

## 3.2.1 Instantiate and Set Properties
The plugin is instantiated with a unique name (`sgie1`) using `Gst.ElementFactory.make`:
```Python
    # Use nvinfer to run inferencing on decoder's output,
    # behaviour of inferencing is set through config file
    pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
    if not pgie:
        sys.stderr.write(" Unable to create pgie \n")

    tracker = Gst.ElementFactory.make("nvtracker", "tracker")
    if not tracker:
        sys.stderr.write(" Unable to create tracker \n")

    sgie1 = Gst.ElementFactory.make("nvinfer", "secondary1-nvinference-engine")
    if not sgie1:
        sys.stderr.write(" Unable to make sgie1 \n")
```

The configuration filename is assigned as a property:
```Python
    #Set properties of pgie and sgie
    pgie.set_property('config-file-path', "dstest2_pgie_config.txt")
    sgie1.set_property('config-file-path', "dstest2_sgie1_config.txt")
```



## 3.2.2 Add to Pipeline and Link
Each plugin is added to the pipeline.  At this point they are unordered in the pipeline bin and not linked:

```Python
    pipeline.add(pgie)
    pipeline.add(tracker)
    pipeline.add(sgie1)
```

Finally, the pipeline plugins are linked together.  Notice how each `.link()` call includes a parameter for the next element in the pipeline.  Now they are in order in the the pipeline

```Python
    streammux.link(pgie)
    pgie.link(tracker)
    tracker.link(sgie1)
    sgie1.link(nvvidconv)
    nvvidconv.link(nvosd)
```

*Note that these snippets are extracted from the longer lists of the complete pipeline.*

## 3.2.3 Configure

In addition to instantiation and pipeline linkage, a configuration file is required for each `Gst-nvinfer` and `Gst-nvtracker` instance.

* [dstest2_pgie_config.txt](deepstream/sources/dli_python_apps/apps/deepstream-test2-rtsp-out-1SGIE/dstest2_pgie_config.txt) configures the same 4-class object detection model you've already worked with, detecting vehicles, bicycles, persons, and road signs.
* [dstest2_sgie1_config.txt](deepstream/sources/dli_python_apps/apps/deepstream-test2-rtsp-out-1SGIE/dstest2_sgie1_config.txt) configures the secondary classification network to determine the color of a vehicle image.  The configuration file specifies, among other things, that __only vehicle objects will be analyzed__. The required parameters for this file are a little different because it is a _classifier_ network, whereas the primary network is a _detector_ network.
* [dstest2_tracker_config.txt](deepstream/sources/dli_python_apps/apps/deepstream-test2-rtsp-out-1SGIE/dstest2_tracker_config.txt) configures the tracking libraries necessary to carry forward the detected images from one frame to the next. The `tracker_config.yml` file is required to further define the low-level library used.

Property definitions for the configuration files can be found in the NVIIDA DeepStream SDK Develope Guide at https://docs.nvidia.com/metropolis/deepstream/dev-guide.
*Note: you will need to copy/paste the address if your Nano is not connected to the Internet directly.*

## 3.2.4 Exercise: Add SGIE2 Plugin to the Pipeline
Create a new app based on `deepstream-test2-rtsp_out-1SGIE` that classifies not only colors, using SGIE1, but also car make, using an additional network, SGIE2.

* You'll need the following specific information about the network model for your configuration file:<br>

   #### Files for the model engine:

```python
model-engine-file=../../../../samples/models/Secondary_CarMake/resnet18.caffemodel_b16_gpu0_fp16.engine
model-file=../../../../samples/models/Secondary_CarMake/resnet18.caffemodel
proto-file=../../../../samples/models/Secondary_CarMake/resnet18.prototxt
mean-file=../../../../samples/models/Secondary_CarMake/mean.ppm
labelfile-path=../../../../samples/models/Secondary_CarMake/labels.txt
int8-calib-file=../../../../samples/models/Secondary_CarMake/cal_trt.bin
```
<!-- **Unique ID**
```c
gie-unique-id=3
``` -->

In [ ]:
# Create a new app located at $MY_APPS/dst2-two-sgie 
#      based on $DLI_APPS/deepstream-test2-rtsp-out-1SGIE
!mkdir -p $MY_APPS/dst2-two-sgie
!cp -rfv $DLI_APPS/deepstream-test2-rtsp-out-1SGIE/* $MY_APPS/dst2-two-sgie

In [ ]:
# Create a new configuration file for SGIE2 
#      based on the one already existing for SGIE1.
!cp -rfv $DLI_APPS/deepstream-test2-rtsp-out-1SGIE/dstest2_sgie1_config.txt \
        $MY_APPS/dst2-two-sgie/dstest2_sgie2_config.txt

* Using what you just learned, modify your new [deepstream_test2_rtsp_out.py](my_apps/dst2-two-sgie/deepstream_test2_rtsp_out.py) and [dstest2_sgie2_config.txt](my_apps/dst2-two-sgie/dstest2_sgie2_config.txt) to add SGIE2 to the pipeline. Then build and run the app to see if it worked!

In [ ]:
# Run the app
!cd $MY_APPS/dst2-two-sgie \
    && python3 deepstream_test2_rtsp_out.py -i $STREAMS/sample_720p.h264

#### How did you do?
If you see something like this image, you did it!  If not, keep trying, or take a peek at the solution code ([Python file](solutions/ex3.2.4_TwoSGIE/dst2-two-sgie/deepstream_test2_rtsp_out.py) and [SGIE2 config file](solutions/ex3.2.4_TwoSGIE/dst2-two-sgie/dstest2_sgie2_config.txt)) in the solutions directory.

<img src="images/02_color_and_make.png" alt="color and make">

# 3.3 Put It All Together
Great job adding SGIE2!  You've learned how to instantiate, link, and configure a multiple network DeepStream application.  Let's take it one step further by adding another network in a new app.

## 3.3.1 Exercise: Three Secondary Networks
Create a new app based on `deepstream-test2-rtsp-out-1SGIE` that classifies colors, makes, and vehicle types. Fill in the following cells with appropriate commands to create and run your app. To edit your files, use the JupyterLab file browser at left to navigate to the correct folder; then, double click on the file you wish to open and edit.

* The configuration information for vehicle _make_ was provided in the previous exercise.  For the vehicle _type_ configuration, you'll need the following:<br>

   #### Files for the model engine:
```python
model-engine-file=../../../../samples/models/Secondary_VehicleTypes/resnet18.caffemodel_b16_gpu0_fp16.engine
model-file=../../../../samples/models/Secondary_VehicleTypes/resnet18.caffemodel
proto-file=../../../../samples/models/Secondary_VehicleTypes/resnet18.prototxt
mean-file=../../../../samples/models/Secondary_VehicleTypes/mean.ppm
labelfile-path=../../../../samples/models/Secondary_VehicleTypes/labels.txt
int8-calib-file=../../../../samples/models/Secondary_VehicleTypes/cal_trt.bin
```
<!-- **Unique ID**
```c
gie-unique-id=4
``` -->

In [ ]:
# TODO
# Create a new app located at $MY_APPS/dst2-three-sgie 
#      based on $DLI_APPS/deepstream-test2-rtsp-out-1SGIE

In [ ]:
# TODO
# Create a new configuration files for SGIOE2 and SGIE3
#      based on the one already existing for SGIE1.

* Modify your new `dst2-three-sgie/deepstream_test2_rtsp_out.py`, and the configuration files as needed. Then run the app to see if it worked!

In [ ]:
# TODO
# Run the app

#### How did you do?
If you see something like this image, you've mastered multiple networks!  If not, keep trying, or take a peek at the [solution code](solutions/ex3.3.1_ThreeSGIE/solution-3.3.1.ipynb) in the solutions directory.  <br>

<img src="images/02_3sgie.png" alt="4 networks">

<h2 style="color:green;">Congratulations!</h2>

You've learned how to add classifier networks to a DeepStream pipeline to create multiple network DeepStream apps.  <br>
Move on to [4.0 Multiple Stream Input](./04_MultiStream.ipynb).


<center><img src="images/DLI Header.png" alt="Header" width="400"></center>